# Debugging

Debugging mode can be turned on globally by setting the `FEEDBAX_DEBUG` environment variable to `"True"`. 

This can be done from within Python, shown below, or by setting `FEEDBAX_DEBUG=True` in your shell before running Python from the command line.

In [ ]:
import os

os.environ["FEEDBAX_DEBUG"] = str(True)

The main debugging feature implemented in Feedbax so far is automatic logging of the state operations performed by any subclass of `AbstractStagedModel`. 

JAX disable JIT?

In [ ]:
jax.config.update("jax_debug_nans", DEBUG)
jax.config.update('jax_disable_jit', DISABLE_JIT)